In [ ]:
import numpy as np
import scipy.stats as stats
from hansolo import *

# np.random.seed(10)

Nb = 100
expert = "EWA"  # "EWA" or "PWA"
learning="blocs" # "blocs": objects presented by random blocs, or "random": each object is randomly chosen with replacement
n_feat=6
N_a = n_feat

m=2501

W_output = np.load(f"{expert}_{learning}_ablationW.npy")[:,:,m,:,:] # indices : j,i,nb,na


M_obj = 500 #size of testing set

dt = 0.002
N = 1000
T = N * dt

freq = 100  # firing rate input neurons +
freq2 = 150  # firing rate input neurons -

p = freq * dt  # spiking proba input neurons
p2 = freq2 * dt

K_input = 12  # nb input neurons
K_output = 2  # nb output neurons


obj = [
    [1, 0, 0, 1, 0, 0],
    [1, 0, 0, 0, 1, 0],
    [1, 0, 0, 0, 0, 1],
    [0, 1, 0, 1, 0, 0],
    [0, 1, 0, 0, 1, 0],
    [0, 1, 0, 0, 0, 1],
    [0, 0, 1, 1, 0, 0],
    [0, 0, 1, 0, 1, 0],
    [0, 0, 1, 0, 0, 1],
]
# blue circle blue square blue triangle red circle red square red triangle gray circle gray square gray triangle

answers = np.zeros((Nb, M_obj, N_a))  # 0 if wrong, 1 if correct

activity = [o + [1 - l for l in o] for o in obj]

for n_a in range(N_a):
    K_input_new = K_input - 2*n_a

    for nb in range(Nb):
    #creation of the list of objects (testing set)
        L=list_objects_random(activity,M_obj)

        feat = np.where(W_output[0,:,nb,n_a] != -1)[0]

        F_input = np.zeros((K_input_new, M_obj))
        F_output=np.zeros((K_output, M_obj))


        for m_obj in range(M_obj):

            activity_cur = L[m_obj]  # current active neurons
            obj_cur = activity_cur[: int(K_input / 2)]  # current object

            activity_cur=activity_cur[feat]
            
            input_neurons=np.zeros((K_input_new, N))
            cur_act = np.where(activity_cur == 1)[0]
            first_split = cur_act[cur_act < (K_input_new // 2)]
            second_split = cur_act[cur_act >= (K_input_new // 2)]
            input_neurons[first_split, :] = Poisson(freq, T, (first_split.size, N))
            input_neurons[second_split, :] = Poisson(freq2, T, (second_split.size, N))

            output_neurons=np.zeros((K_output, N))

            # simulation of the output neurons
        
            
            for i in range(K_output):
                #print(np.shape(input_neurons))
                #print(np.shape(W_output[i, feat, nb, n_a]))
                output_neurons[i, :] = Hawkes_lin(input_neurons, W_output[i, feat, nb, n_a])

            F_input[:, m_obj] = np.sum(input_neurons, axis=1) / T
            F_output[:, m_obj] = np.sum(output_neurons, axis=1) / T

            
            if (in_A(obj_cur) and F_output[0, m_obj] > F_output[1, m_obj]) or (
                in_B(obj_cur) and F_output[1, m_obj] > F_output[0, m_obj]
            ):
                answers[nb, m_obj, n_a] = 1
            
    print(n_a)


np.save(f"{expert}_{learning}_test_ablation", answers)




In [ ]:
import numpy as np
Nb = 100
expert = "PWA"  # "EWA" or "PWA"
learning="blocs" # "blocs": objects presented by random blocs, or "random": each object is randomly chosen with replacement
n_feat=6
N_a = n_feat

m=2501

W_output = np.load(f"{expert}_{learning}_ablationW.npy")[:,:,m,:,:] # indices : j,i,nb,na
n_a = 1
nb=0

print(W_output[0,:,nb,n_a])
feat = np.where(W_output[0,:,nb,n_a] != -1)[0]
print(feat)

